### import

In [1]:
from os.path import join
import pandas as pd
import os, re
import functions

### 파일읽기

In [2]:
# 필요 데이터 읽기
##################################
company = "솔림"      # 솔림 or 대성
basedate = "250228" 
##################################
path_raw_data = functions.키워드로파일명찾기(join(r"D:\3.자산\전산 dataset", company, basedate, "pkl"), "전체_원데이터")
path_grt = functions.키워드로파일명찾기(join(r"D:\3.자산\전산 dataset", company, basedate, "pkl"), "보증인새창.*새보증인상태")
raw_data = pd.read_pickle(path_raw_data)
grt = pd.read_pickle(path_grt)

# 계좌, 보증인 종결건 제외하기
raw_data = raw_data.query('채권상태!="종료"')
grt = grt.query('새보증인상태!="종료"')

In [3]:
# 검수할 새채무자키 읽기
checklist = pd.read_excel(join(r"C:\Users\DATA\Desktop\checklist.xlsx"))
print(len(checklist))
# 중복제거
checklist = checklist.drop_duplicates(subset=["새채무자키"])
print(len(checklist))

42
38


### 파일작성

In [4]:
# 체크리스트와 계좌병합하기
merged = checklist.merge(raw_data[["새채무자키","채무자키","계좌키","성명","원채권사","매각사구분","반영OPB","시효완성월","시효완성일","시효사유" ]], how='left')
# 컬럼명 변경
merged = merged.rename(columns={"성명":"채무자명"})
# merged["차주명"] = merged["채무자명"]
# 보증인키 칼럼 추가
merged["보증인키"] = ""
merged.insert(3,'보증인키', merged.pop('보증인키'))
print('병합후 :',len(merged))

병합후 : 42


In [5]:
# merged와 칼럼이 같은 새로운 보증인df 만들기
grt_for_merged = grt[["계좌키","보증인키","보증인성명인"]]
grt_for_merged = grt_for_merged.rename(columns={'보증인성명인':'채무자명'})
    # 새채무자키 입히기
grt_for_merged = grt_for_merged.merge(merged, on=["계좌키"], how='right', suffixes=['','_차주'])
    # 차주행 지우고, 채무자명_차주 열 지우기
grt_for_merged = grt_for_merged.dropna(subset=["보증인키"])
    # merged와 칼럼순서 맞추기
grt_for_merged = grt_for_merged[merged.columns.to_list()]

In [6]:
# 차주 + 보증인 리스트
merged2 = pd.concat([merged, grt_for_merged], ignore_index=True).sort_values(by=["새채무자키","보증인키", "채무자명","계좌키"]).reset_index(drop=True)

In [7]:
# 추가 열
merged2.insert(0,'실사일', '')
for i in range(17) :
    merged2[f'cols{i+1}'] = ''

In [8]:
# 엑셀저장 ###################################
fn_write = "검수리스트_무차입_250228_실효폐지등"
#############################################
functions.save_df_to_excel_two_underline(merged2, join(r"C:\Users\DATA\Desktop",fn_write+".xlsx"), key_columns_no=2, sub_key_columns_no=6) 